# NY Flight Prediction

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import bokeh as bkh
import matplotlib.pyplot as plt

In [2]:
NYF_data=pd.read_csv('Dataset/flight_data.csv')

In [3]:
NYF_data.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,01-01-2013 05:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,01-01-2013 05:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,01-01-2013 05:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,01-01-2013 05:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,01-01-2013 06:00


In [4]:
NYF_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 19 columns):
year              336776 non-null int64
month             336776 non-null int64
day               336776 non-null int64
dep_time          328521 non-null float64
sched_dep_time    336776 non-null int64
dep_delay         328521 non-null float64
arr_time          328063 non-null float64
sched_arr_time    336776 non-null int64
arr_delay         327346 non-null float64
carrier           336776 non-null object
flight            336776 non-null int64
tailnum           334264 non-null object
origin            336776 non-null object
dest              336776 non-null object
air_time          327346 non-null float64
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
dtypes: float64(5), int64(9), object(5)
memory usage: 42.4+ MB


In [30]:
Dep_time=pd.pivot_table(NYF_data,values='dep_time',index=['origin'],columns=['carrier'],aggfunc=np.median)
Dep_time=round(Dep_time,0)
Dep_time

carrier,9E,AA,AS,B6,DL,EV,F9,FL,HA,MQ,OO,UA,US,VX,WN,YV
origin,,,,,,,,,,,,,,,,
EWR,1255.0,1425.0,1805.0,1314.0,1301.0,1400.0,NaN,NaN,NaN,1329.0,1534.0,1413.0,1032.0,1251.0,1300.0,NaN
JFK,1610.0,1456.0,NaN,1432.0,1552.0,1354.0,NaN,NaN,954.0,1525.0,NaN,1153.0,1025.0,1148.0,NaN,NaN
LGA,1432.0,1239.0,NaN,1328.0,1300.0,1439.0,1719.0,1418.0,NaN,1348.0,1759.0,1156.0,1257.0,NaN,1238.0,1636.0


In [31]:
Arr_time=pd.pivot_table(NYF_data,values='arr_time',index=['origin'],columns=['carrier'],aggfunc=np.median)
Arr_time=round(Arr_time,0)
Arr_time

carrier,9E,AA,AS,B6,DL,EV,F9,FL,HA,MQ,OO,UA,US,VX,WN,YV
origin,,,,,,,,,,,,,,,,
EWR,1457.0,1650.0,2043.0,1417.0,1515.0,1519.0,NaN,NaN,NaN,1442.0,1756.0,1607.0,1328.0,1434.0,1510.0,NaN
JFK,1801.0,1705.0,NaN,1429.0,1827.0,1502.0,NaN,NaN,1448.0,1734.0,NaN,1434.0,1240.0,1358.0,NaN,NaN
LGA,1607.0,1501.0,NaN,1411.0,1530.0,1602.0,1927.0,1612.0,NaN,1526.0,1945.0,1348.0,1441.0,NaN,1416.0,1800.0


In [7]:
NYF_data[NYF_data.dep_time.isnull()]

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
838,2013,1,1,NaN,1630,NaN,NaN,1815,NaN,EV,4308,N18120,EWR,RDU,NaN,416,16,30,01-01-2013 16:00
839,2013,1,1,NaN,1935,NaN,NaN,2240,NaN,AA,791,N3EHAA,LGA,DFW,NaN,1389,19,35,01-01-2013 19:00
840,2013,1,1,NaN,1500,NaN,NaN,1825,NaN,AA,1925,N3EVAA,LGA,MIA,NaN,1096,15,0,01-01-2013 15:00
841,2013,1,1,NaN,600,NaN,NaN,901,NaN,B6,125,N618JB,JFK,FLL,NaN,1069,6,0,01-01-2013 06:00
1777,2013,1,2,NaN,1540,NaN,NaN,1747,NaN,EV,4352,N10575,EWR,CVG,NaN,569,15,40,02-01-2013 15:00
1778,2013,1,2,NaN,1620,NaN,NaN,1746,NaN,EV,4406,N13949,EWR,PIT,NaN,319,16,20,02-01-2013 16:00
1779,2013,1,2,NaN,1355,NaN,NaN,1459,NaN,EV,4434,N10575,EWR,MHT,NaN,209,13,55,02-01-2013 13:00
1780,2013,1,2,NaN,1420,NaN,NaN,1644,NaN,EV,4935,N759EV,EWR,ATL,NaN,746,14,20,02-01-2013 14:00
1781,2013,1,2,NaN,1321,NaN,NaN,1536,NaN,EV,3849,N13550,EWR,IND,NaN,645,13,21,02-01-2013 13:00
1782,2013,1,2,NaN,1545,NaN,NaN,1910,NaN,AA,133,NaN,JFK,LAX,NaN,2475,15,45,02-01-2013 15:00


In [8]:
NYF_data.loc[NYF_data['dep_time'].isnull(),'dep_time']=NYF_data.loc[NYF_data['dep_time'].isnull(),'sched_dep_time']

In [9]:
NYF_data[NYF_data.arr_time.isnull()]

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
754,2013,1,1,2016.0,1930,46.0,NaN,2220,NaN,EV,4204,N14168,EWR,OKC,NaN,1325,19,30,01-01-2013 19:00
838,2013,1,1,1630.0,1630,NaN,NaN,1815,NaN,EV,4308,N18120,EWR,RDU,NaN,416,16,30,01-01-2013 16:00
839,2013,1,1,1935.0,1935,NaN,NaN,2240,NaN,AA,791,N3EHAA,LGA,DFW,NaN,1389,19,35,01-01-2013 19:00
840,2013,1,1,1500.0,1500,NaN,NaN,1825,NaN,AA,1925,N3EVAA,LGA,MIA,NaN,1096,15,0,01-01-2013 15:00
841,2013,1,1,600.0,600,NaN,NaN,901,NaN,B6,125,N618JB,JFK,FLL,NaN,1069,6,0,01-01-2013 06:00
1714,2013,1,2,2041.0,2045,-4.0,NaN,2359,NaN,B6,147,N630JB,JFK,RSW,NaN,1074,20,45,02-01-2013 20:00
1756,2013,1,2,2145.0,2129,16.0,NaN,33,NaN,UA,1299,N12221,EWR,RSW,NaN,1068,21,29,02-01-2013 21:00
1777,2013,1,2,1540.0,1540,NaN,NaN,1747,NaN,EV,4352,N10575,EWR,CVG,NaN,569,15,40,02-01-2013 15:00
1778,2013,1,2,1620.0,1620,NaN,NaN,1746,NaN,EV,4406,N13949,EWR,PIT,NaN,319,16,20,02-01-2013 16:00
1779,2013,1,2,1355.0,1355,NaN,NaN,1459,NaN,EV,4434,N10575,EWR,MHT,NaN,209,13,55,02-01-2013 13:00


In [10]:
NYF_data.loc[NYF_data['arr_time'].isnull(),'arr_time']=NYF_data.loc[NYF_data['arr_time'].isnull(),'sched_arr_time']

In [11]:
NYF_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 19 columns):
year              336776 non-null int64
month             336776 non-null int64
day               336776 non-null int64
dep_time          336776 non-null float64
sched_dep_time    336776 non-null int64
dep_delay         328521 non-null float64
arr_time          336776 non-null float64
sched_arr_time    336776 non-null int64
arr_delay         327346 non-null float64
carrier           336776 non-null object
flight            336776 non-null int64
tailnum           334264 non-null object
origin            336776 non-null object
dest              336776 non-null object
air_time          327346 non-null float64
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
dtypes: float64(5), int64(9), object(5)
memory usage: 42.4+ MB


In [12]:
NYF_data[NYF_data.dep_delay.isnull()]

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
838,2013,1,1,1630.0,1630,NaN,1815.0,1815,NaN,EV,4308,N18120,EWR,RDU,NaN,416,16,30,01-01-2013 16:00
839,2013,1,1,1935.0,1935,NaN,2240.0,2240,NaN,AA,791,N3EHAA,LGA,DFW,NaN,1389,19,35,01-01-2013 19:00
840,2013,1,1,1500.0,1500,NaN,1825.0,1825,NaN,AA,1925,N3EVAA,LGA,MIA,NaN,1096,15,0,01-01-2013 15:00
841,2013,1,1,600.0,600,NaN,901.0,901,NaN,B6,125,N618JB,JFK,FLL,NaN,1069,6,0,01-01-2013 06:00
1777,2013,1,2,1540.0,1540,NaN,1747.0,1747,NaN,EV,4352,N10575,EWR,CVG,NaN,569,15,40,02-01-2013 15:00
1778,2013,1,2,1620.0,1620,NaN,1746.0,1746,NaN,EV,4406,N13949,EWR,PIT,NaN,319,16,20,02-01-2013 16:00
1779,2013,1,2,1355.0,1355,NaN,1459.0,1459,NaN,EV,4434,N10575,EWR,MHT,NaN,209,13,55,02-01-2013 13:00
1780,2013,1,2,1420.0,1420,NaN,1644.0,1644,NaN,EV,4935,N759EV,EWR,ATL,NaN,746,14,20,02-01-2013 14:00
1781,2013,1,2,1321.0,1321,NaN,1536.0,1536,NaN,EV,3849,N13550,EWR,IND,NaN,645,13,21,02-01-2013 13:00
1782,2013,1,2,1545.0,1545,NaN,1910.0,1910,NaN,AA,133,NaN,JFK,LAX,NaN,2475,15,45,02-01-2013 15:00


In [18]:
NYF_data.loc[NYF_data['dep_delay'].isnull(),'dep_delay']=0

In [19]:
NYF_data[NYF_data.arr_delay.isnull()]

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
471,2013,1,1,1525.0,1530,-5.0,1934.0,1805,NaN,MQ,4525,N719MQ,LGA,XNA,NaN,1147,15,30,01-01-2013 15:00
477,2013,1,1,1528.0,1459,29.0,2002.0,1647,NaN,EV,3806,N17108,EWR,STL,NaN,872,14,59,01-01-2013 14:00
615,2013,1,1,1740.0,1745,-5.0,2158.0,2020,NaN,MQ,4413,N739MQ,LGA,XNA,NaN,1147,17,45,01-01-2013 17:00
643,2013,1,1,1807.0,1738,29.0,2251.0,2103,NaN,UA,1228,N31412,EWR,SAN,NaN,2425,17,38,01-01-2013 17:00
725,2013,1,1,1939.0,1840,59.0,29.0,2151,NaN,9E,3325,N905XJ,JFK,DFW,NaN,1391,18,40,01-01-2013 18:00
733,2013,1,1,1952.0,1930,22.0,2358.0,2207,NaN,EV,4333,N11194,EWR,TUL,NaN,1215,19,30,01-01-2013 19:00
754,2013,1,1,2016.0,1930,46.0,2220.0,2220,NaN,EV,4204,N14168,EWR,OKC,NaN,1325,19,30,01-01-2013 19:00
838,2013,1,1,1630.0,1630,0.0,1815.0,1815,NaN,EV,4308,N18120,EWR,RDU,NaN,416,16,30,01-01-2013 16:00
839,2013,1,1,1935.0,1935,0.0,2240.0,2240,NaN,AA,791,N3EHAA,LGA,DFW,NaN,1389,19,35,01-01-2013 19:00
840,2013,1,1,1500.0,1500,0.0,1825.0,1825,NaN,AA,1925,N3EVAA,LGA,MIA,NaN,1096,15,0,01-01-2013 15:00


In [29]:
NYF_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 19 columns):
year              336776 non-null int64
month             336776 non-null int64
day               336776 non-null int64
dep_time          336776 non-null int64
sched_dep_time    336776 non-null int64
dep_delay         336776 non-null int64
arr_time          336776 non-null int64
sched_arr_time    336776 non-null int64
arr_delay         327346 non-null float64
carrier           336776 non-null object
flight            336776 non-null int64
tailnum           334264 non-null object
origin            336776 non-null object
dest              336776 non-null object
air_time          327346 non-null float64
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
dtypes: float64(2), int64(12), object(5)
memory usage: 42.4+ MB


In [ ]:
NYF_data.info()

In [ ]:
NYF_data[NYF_data.sched_arr_time==NYF_data.arr_time][NYF_data.arr_delay.isnull()]

In [ ]:
NYF_data['arr_delay']=NYF_data.arr_time - NYF_data.sched_arr_time

In [ ]:
NYF_data=NYF_data.drop('tailnum',axis=1)

In [ ]:
NYF_data[NYF_data.air_time.isnull()]

In [ ]:
NYF_data.dep_time=NYF_data.dep_time.astype(np.int64)
NYF_data.dep_delay=NYF_data.dep_delay.astype(np.int64)
NYF_data.arr_time=NYF_data.arr_time.astype(np.int64)
NYF_data.arr_delay=NYF_data.arr_delay.astype(np.int64)

In [ ]:
NYF_data

In [ ]:
NYF_data.info()